# Generate demo data
The point of this notebook is to test and implement the generation of demo data. The data will be used to serve a real-world search query to the frontend in development environments that are disconnected from the Mongo server.

## Generation of demo data

In [1]:
import sys
sys.path.append('../backend/lib')
sys.path.append('../backend')
from GraphSession import GraphSession
from ResultGraph import ResultGraph
from networkx import generate_gml
import networkx as nx

search_input = "campbell cancer leukemia" #"nlp genomics"
graphsession = GraphSession(search_input)
citations = graphsession.get_citations_from_fulltext_mongo(search_input, retmax=200)
resultGraph = ResultGraph()
resultGraph.populate_from_cite_dict(citations)
        
resultGraph.extract_by_connectivity(connectivity=1)
resultGraph.extract_by_connectivity(connectivity=0)
        
# Query metadata
metadataList = graphsession.get_metadataList_from_mongo(resultGraph.nodeIds)
resultGraph.add_metadata_to_graph(metadataList)


05/13/2018 01:59:45 PM [INFO]: RequestFullTextSearch: Creating request
05/13/2018 01:59:45 PM [INFO]: ConnectEutils: Connecting to eutils API:
 https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pmc&tool=graphsearch&sort=relevance&term=campbell+cancer+leukemia&retmax=200
05/13/2018 01:59:46 PM [INFO]: MongoSession: Mongo db connection successful
05/13/2018 01:59:47 PM [INFO]: MongoSession: Mongo db connection successful


In [2]:
from networkx.readwrite import json_graph
import json

data = json_graph.node_link_data(resultGraph.G)
json.dump(data,open('output/demo_network.json','w'),indent=2)

### Try reimporting

In [9]:
from networkx.readwrite import json_graph
import json

def read_json_file(filename):
    with open(filename) as f:
        js_graph = json.load(f)
    return json_graph.node_link_graph(js_graph)

read_json_file('output/demo_network.json')